In [1]:
import pymorphy2
import pandas as pd
import numpy as np
import os

In [2]:
morph = pymorphy2.MorphAnalyzer()
INPUT_PATH = r'../input/norm_challenge_ru'


In [3]:
print(morph.parse('пять'))
print(morph.parse('пяти'))
print(morph.parse('пятью'))
print(morph.parse('пятого'))
print(morph.parse('пятерых'))
print(morph.parse('пятый'))
print(morph.parse('пятых'))
# print(morph.parse('ста'), morph.parse('двадцати'))
# print(morph.parse('сорока одного'))
# print(morph.parse('двадцати'), morph.parse('двух'))
# print(morph.parse('девяноста'))

[Parse(word='пять', tag=OpencorporaTag('NUMR nomn'), normal_form='пять', score=0.5, methods_stack=((<DictionaryAnalyzer>, 'пять', 889, 0),)), Parse(word='пять', tag=OpencorporaTag('NUMR accs'), normal_form='пять', score=0.5, methods_stack=((<DictionaryAnalyzer>, 'пять', 889, 3),))]
[Parse(word='пяти', tag=OpencorporaTag('NUMR gent'), normal_form='пять', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'пяти', 889, 1),)), Parse(word='пяти', tag=OpencorporaTag('NUMR datv'), normal_form='пять', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'пяти', 889, 2),)), Parse(word='пяти', tag=OpencorporaTag('NUMR loct'), normal_form='пять', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'пяти', 889, 5),))]
[Parse(word='пятью', tag=OpencorporaTag('NUMR ablt'), normal_form='пять', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'пятью', 889, 4),))]
[Parse(word='пятого', tag=OpencorporaTag('ADJF,Anum neut,sing,gent'), normal_form='пять', score=0.6, met

In [ ]:

train = pd.read_csv(os.path.join(INPUT_PATH, 'ru_train.csv'), encoding='utf-8', index_col=False)
#train['before'] = train['before'].str.lower()
#train['after'] = train['after'].str.lower()
train['after_wc'] = train['after'].map(lambda x: len(str(x).split()))
train['before_prevprev'] = train['before'].shift(2)
train['before_prev'] = train['before'].shift(1)
train['before_next'] = train['before'].shift(-1)
train['before_nextnext'] = train['before'].shift(-2)
#train = train[~(train['before'] == train['after'])]

#train[~(train['class'] == 'LETTERS') & (train['after_c'] > 4)]
#train = train.groupby(['before', 'after'], as_index=False)['sentence_id'].count()
#train = train.sort_values(['sentence_id','before'], ascending=[False, True])
#train = train.drop_duplicates(['before'])
#d = {key: value for (key, value) in train[['before', 'after']].values}


In [3]:
train.head()

,sentence_id,token_id,class,before,after,after_wc,before_prevprev,before_prev,before_next,before_nextnext
0,0,0,PLAIN,По,По,1,NaN,NaN,состоянию,на
1,0,1,PLAIN,состоянию,состоянию,1,NaN,По,на,1862 год
2,0,2,PLAIN,на,на,1,По,состоянию,1862 год,.
3,0,3,DATE,1862 год,тысяча восемьсот шестьдесят второй год,5,состоянию,на,.,Оснащались
4,0,4,PUNCT,.,.,1,на,1862 год,Оснащались,латными


In [4]:
train[(train['before'] == train['after'])]['class'].value_counts()

PLAIN         6816337
PUNCT         2288640
VERBATIM       144968
MEASURE           614
CARDINAL          508
ORDINAL           203
ELECTRONIC        155
DECIMAL           104
DATE              100
MONEY              67
TIME               50
LETTERS            47
FRACTION           25
Name: class, dtype: int64

In [5]:
train[~(train['before'] == train['after'])]['class'].value_counts()

PLAIN         544102
CARDINAL      271934
LETTERS       189481
DATE          185859
ORDINAL        46535
MEASURE        39920
VERBATIM       12944
TELEPHONE      10088
DECIMAL         7193
ELECTRONIC      5677
MONEY           2623
FRACTION        2435
DIGIT           2012
TIME            1895
Name: class, dtype: int64

In [ ]:
train[(train['class']=='VERBATIM') & (train['before'].fillna('').str.contains('\d'))]['after'].head(10)



In [ ]:
train[(train['class'] == 'VERBATIM') & ~(train['before'] == train['after'])]['after'].value_counts()


In [12]:
context = morph.parse('века')
context

[Parse(word='века', tag=OpencorporaTag('NOUN,inan,masc sing,gent'), normal_form='век', score=0.625, methods_stack=((<DictionaryAnalyzer>, 'века', 666, 1),)),
 Parse(word='века', tag=OpencorporaTag('NOUN,inan,neut sing,gent'), normal_form='веко', score=0.125, methods_stack=((<DictionaryAnalyzer>, 'века', 556, 1),)),
 Parse(word='века', tag=OpencorporaTag('NOUN,inan,masc plur,nomn'), normal_form='век', score=0.125, methods_stack=((<DictionaryAnalyzer>, 'века', 666, 7),)),
 Parse(word='века', tag=OpencorporaTag('NOUN,inan,masc plur,accs'), normal_form='век', score=0.125, methods_stack=((<DictionaryAnalyzer>, 'века', 666, 10),))]

In [10]:
context[0].tag

OpencorporaTag('NOUN,inan,masc sing,gent')

In [46]:
morph.parse('девятнадцатого')

[Parse(word='девятнадцатого', tag=OpencorporaTag('ADJF,Anum masc,sing,gent'), normal_form='девятнадцать', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'девятнадцатого', 889, 7),)),
 Parse(word='девятнадцатого', tag=OpencorporaTag('ADJF,Anum anim,masc,sing,accs'), normal_form='девятнадцать', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'девятнадцатого', 889, 10),)),
 Parse(word='девятнадцатого', tag=OpencorporaTag('ADJF,Anum neut,sing,gent'), normal_form='девятнадцать', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'девятнадцатого', 889, 20),))]

In [ ]:
test = pd.read_csv('../input/ru_test.csv')
test['id'] = test['sentence_id'].astype(str) + '_' + test['token_id'].astype(str)
test['before_l'] = test['before'].str.lower()
test['after'] = test['before_l'].map(lambda x: d[x] if x in d else x) #should use same case as original before :)


def fcase(obefore, lbefore, after):
    if lbefore == after:
        return obefore
    else:
        return after
test['after'] = test.apply(lambda r: fcase(r['before'],r['before_l'],r['after']), axis=1)

test[['id','after']].to_csv('submission.csv', index=False)